# Notebook to compare flux diagrams for 2 different mechanisms

In [1]:
import os
import sys
import numpy as np
import pydot
import rmgpy.chemkin
import rmgpy.tools.fluxdiagram
import copy

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


### Settings

In [2]:
# Options controlling the individual flux diagram renderings:
program = 'dot'  # The program to use to lay out the nodes and edges
max_node_count = 50  # The maximum number of nodes to show in the diagram
max_edge_count = 50  # The maximum number of edges to show in the diagram
concentration_tol = 1e-6  # The lowest fractional concentration to show (values below this will appear as zero)
species_rate_tol = 1e-6  # The lowest fractional species rate to show (values below this will appear as zero)
max_node_pen_width = 7.0  # The thickness of the border around a node at maximum concentration
max_edge_pen_width = 9.0  # The thickness of the edge at maximum species rate


### Load Mechanisms - must get lists of RMG formatted species and reactions

In [3]:
diagram_base_name = 'example_flux_comparison'  # where to save the flux diagrams
os.makedirs(diagram_base_name, exist_ok=True)

species_image_path = './species_images'  # where to get/place the images of each species later on
os.makedirs(species_image_path, exist_ok=True)


# --------------------------------- EDIT THIS ----------------------------
mech_1_inp = './ethane_oxidation/chem_annotated.inp'
mech_1_dict = './ethane_oxidation/species_dictionary.txt'
mech_1_label = 'Ethane Oxidation'
t1_ignition = 0.06535855787324196  # <---------- script will try to offset flux diagram time index so that these times match


# --------------------------------- EDIT THIS ----------------------------
mech_2_inp = './minimal/chem_annotated.inp'
mech_2_dict = './minimal/species_dictionary.txt'
mech_2_label = 'ethane pyrolysis'
t2_ignition = 0.06535855787324196   # <---------- script will try to offset flux diagram time index so that these times match


### Load Mechanisms Option A: using RMG's simple solver

In [ ]:
rmg_input_file = './minimal/input.py'  # for simulation conditions


generate_images = False
print('Loading RMG job 1...')
rmg_job1 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_1_inp,
    mech_1_dict,
    generate_images=generate_images,
    check_duplicates=True
)

print('Loading RMG job 2...')
rmg_job2 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_2_inp,
    mech_2_dict,
    generate_images=generate_images,
    check_duplicates=True
)


# run the simulation to get times1, concentrations1, reaction_rates1, and then times2, concentrations2, and reaction_rates2
print('Conducting simulation of reaction 1')
times1, concentrations1, reaction_rates1 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job1.reaction_model,
    rmg_job1.reaction_systems[0],
)

print('Conducting simulation of reaction 2')
times2, concentrations2, reaction_rates2 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job2.reaction_model,
    rmg_job2.reaction_systems[0],
)

# Get the RMG species and reactions objects
species_list1 = rmg_job1.reaction_model.core.species[:]
reaction_list1 = rmg_job1.reaction_model.core.reactions[:]
num_species1 = len(species_list1)

species_list2 = rmg_job2.reaction_model.core.species[:]
reaction_list2 = rmg_job2.reaction_model.core.reactions[:]
num_species2 = len(species_list2)

### Load Mechanisms Option B: using saved .npy files from Cantera/RMS
see save_flux_rates.ipynb for example of saving these

In [13]:
# Still have to load the mechanisms to get the lists of species and reactions
species_list1, reaction_list1 = rmgpy.chemkin.load_chemkin_file(mech_1_inp, mech_1_dict)
species_list2, reaction_list2 = rmgpy.chemkin.load_chemkin_file(mech_2_inp, mech_2_dict)

num_species1 = len(species_list1)
num_species2 = len(species_list2)


times1 = np.load('./ethane_oxidation/times.npy')
concentrations1 = np.load('./ethane_oxidation/concentrations.npy')
reaction_rates1 = np.load('./ethane_oxidation/rates.npy')

times2 = np.load('./minimal/times.npy')
concentrations2 = np.load('./minimal/concentrations.npy')
reaction_rates2 = np.load('./minimal/rates.npy')

assert concentrations1.shape[1] == num_species1
assert concentrations2.shape[1] == num_species2

# you have to come up with your own mapping if the number of cantera reactions doesn't match the number of RMG reactions
assert reaction_rates1.shape[1] == len(reaction_list1)
assert reaction_rates2.shape[1] == len(reaction_list2)


# Construct the big matrix of fluxes from one species to another

In [5]:
# Compute the rates between each pair of species (build up big matrices)
species_rates1 = np.zeros((len(times1), num_species1, num_species1), float)
for index1, reaction1 in enumerate(reaction_list1):
    rate1 = reaction_rates1[:, index1]
    if not reaction1.pairs: reaction1.generate_pairs()
    for reactant1, product1 in reaction1.pairs:
        reactant_index1 = species_list1.index(reactant1)
        product_index1 = species_list1.index(product1)
        species_rates1[:, reactant_index1, product_index1] += rate1
        species_rates1[:, product_index1, reactant_index1] -= rate1
        
species_rates2 = np.zeros((len(times2), num_species2, num_species2), float)
for index2, reaction2 in enumerate(reaction_list2):
    rate2 = reaction_rates2[:, index2]
    if not reaction2.pairs: reaction2.generate_pairs()
    for reactant2, product2 in reaction2.pairs:
        reactant_index2 = species_list2.index(reactant2)
        product_index2 = species_list2.index(product2)
        species_rates2[:, reactant_index2, product_index2] += rate2
        species_rates2[:, product_index2, reactant_index2] -= rate2

### Scale the flux concentrations according to the maximum across all time
Your particular system might call for a different approach

In [6]:
# Determine the maximum concentration for each species and the maximum overall concentration
max_concentrations1 = np.max(np.abs(concentrations1), axis=0)
max_concentration1 = np.max(max_concentrations1)

# Determine the maximum reaction rates
max_reaction_rates1 = np.max(np.abs(reaction_rates1), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates1 = np.max(np.abs(species_rates1), axis=0)
max_species_rate1 = np.max(max_species_rates1)
species_index1 = max_species_rates1.reshape((num_species1 * num_species1)).argsort()


max_concentrations2 = np.max(np.abs(concentrations2), axis=0)
max_concentration2 = np.max(max_concentrations2)

# Determine the maximum reaction rates
max_reaction_rates2 = np.max(np.abs(reaction_rates2), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates2 = np.max(np.abs(species_rates2), axis=0)
max_species_rate2 = np.max(max_species_rates2)
species_index2 = max_species_rates2.reshape((num_species2 * num_species2)).argsort()


max_species_rate_total = max(max_species_rate1, max_species_rate2)
max_concentration_total = max(max_concentration1, max_concentration2)

### Determine nodes and edges to include for each model

In [7]:
nodes1 = []
edges1 = []
for i in range(num_species1 * num_species1):
    product_index1, reactant_index1 = divmod(species_index1[-i - 1], num_species1)
    if reactant_index1 > product_index1:
        # Both reactant -> product and product -> reactant are in this list, so only keep one of them
        continue
    if max_species_rates1[reactant_index1, product_index1] == 0:
        break
    if reactant_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(reactant_index1)
    if product_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(product_index1)
    if [reactant_index1, product_index1] not in edges1 and [product_index1, reactant_index1] not in edges1:
        edges1.append([reactant_index1, product_index1])
    if len(nodes1) > max_node_count:
        break
    if len(edges1) >= max_edge_count:
        break
        
nodes2 = []
edges2 = []
for i in range(num_species2 * num_species2):
    product_index2, reactant_index2 = divmod(species_index2[-i - 1], num_species2)
    if reactant_index2 > product_index2:
        # Both reactant -> product and product -> reactant are in this list, so only keep one of them
        continue
    if max_species_rates2[reactant_index2, product_index2] == 0:
        break
    if reactant_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(reactant_index2)
    if product_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(product_index2)
    if [reactant_index2, product_index2] not in edges2 and [product_index2, reactant_index2] not in edges2:
        edges2.append([reactant_index2, product_index2])
    if len(nodes2) > max_node_count:
        break
    if len(edges2) >= max_edge_count:
        break
  

### create mapping between models

In [8]:
species2_to_1 = {}
species2_to_1 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list2)):
    for j in range(len(species_list1)):
        if species_list2[i].is_isomorphic(species_list1[j]):
            species2_to_1[i] = j
            break
    else:
        species2_to_1[i] = -1

species1_to_2 = {}
species1_to_2 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list1)):
    for j in range(len(species_list2)):
        if species_list1[i].is_isomorphic(species_list2[j]):
            species1_to_2[i] = j
            break
    else:
        species1_to_2[i] = -1


In [9]:
# Function to grab and generate the image for a given species
def get_image_path(species):
    species_index = str(species) + '.png'
    image_path = ''
    if not species_image_path or not os.path.exists(species_image_path):  # species_image_path is defined while loading the mechanism
        raise OSError
    for root, dirs, files in os.walk(species_image_path):
        for f in files:
            if f == species_index:
                image_path = os.path.join(root, f)
                break
    if not image_path:
        image_path = os.path.join(species_image_path, species_index)
        species.molecule[0].draw(image_path)
    return os.path.abspath(image_path)

### Build the graph

In [10]:
# Create the graph
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
color1 = colors[0]
color2 = colors[1]

# Grab the fluxes from the time closest (without going over) to official ignition delay time
t1 = np.abs(times1 - t1_ignition).argmin()
if times1[t1] > t1_ignition:
    t1 -= 1
t2 = np.abs(times2 - t2_ignition).argmin()
if times2[t2] > t2_ignition:
    t2 -= 1


label_automatically = False  # Turned off because manual labels upset the graph placements less
assert -1 not in nodes1
assert -1 not in nodes2


for t in range(len(times1))[::2]:  # downsample by 2 to create fewer files
    t1 = t
    t2 = t
    if t2 >= len(times2):
        break
    slope = -max_node_pen_width / np.log10(concentration_tol)
    graph = pydot.Dot('flux_diagram', graph_type='digraph', overlap="false")
    graph.set_fontname('sans')
    graph.set_fontsize('10')

    # ----------------------------ADD NODES ------------------------------#
    # For Mechanism 1
    for index1 in nodes1:
        nodewidths = np.zeros(3)  # keep track of species concentrations/nodewidths for all 3 mechanisms
        species1 = species_list1[index1]
        node1 = pydot.Node(name=str(species1))
        concentration1 = concentrations1[t1, index1] / max_concentration_total
        if concentration1 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration1) + max_node_pen_width, 3)
            nodewidths[0] = penwidth
        node1.set_penwidth(penwidth)
        node1.set_fillcolor('white')
        node1.set_color(color1)
        image_path1 = get_image_path(species1)
        if os.path.exists(image_path1):
            node1.set_image(image_path1)
            node1.set_label("")

        index2 = species1_to_2[index1]
        if index2 >= 0:
            concentration2 = concentrations2[t2, index2] / max_concentration_total
            if concentration2 < concentration_tol:
                penwidth = 0.0
            else:
                penwidth = round(slope * np.log10(concentration2) + max_node_pen_width, 3)
                nodewidths[1] = penwidth
                if node1.get_penwidth() > 0:
                    node1.set_color('black')
                else:
                    node1.set_color(color2)
                    node1.set_penwidth(penwidth)


        if node1.get_color() == 'black':
            node1.set_penwidth(np.average(nodewidths))
        graph.add_node(node1)

    # For Mechanism 2
    for index2 in nodes2:
        if species2_to_1[index2] in nodes1:
            continue  # already took care of it above

        nodewidths = np.zeros(3)
        species2 = species_list2[index2]
        node2 = pydot.Node(name=str(species2))
        concentration2 = concentrations2[t2, index2] / max_concentration_total
        if concentration2 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration2) + max_node_pen_width, 3)
            nodewidths[1] = penwidth
        node2.set_fillcolor('white')
        node2.set_color(color2)
        node2.set_penwidth(penwidth)
        # Try to use an image instead of the label
        image_path2 = get_image_path(species2)
        if os.path.exists(image_path2):
            node2.set_image(image_path2)
            node2.set_label("")


        if node2.get_color() == 'black':
            node2.set_penwidth(np.average(nodewidths))

        graph.add_node(node2)



    # ------------------------------- EDGES ------------------------------#
    # Add an edge for each species-species rate
    slope = -max_edge_pen_width / np.log10(species_rate_tol)

    # Go through edges in Mechanism 1
    for reactant_index1, product_index1 in edges1:
        if reactant_index1 in nodes1 and product_index1 in nodes1:
            reactant1 = species_list1[reactant_index1]
            product1 = species_list1[product_index1]
            label1 = ''

            edge1 = pydot.Edge(str(reactant1), str(product1), color=color1)
            species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate_total
            if species_rate1 < 0:
                edge1.set_dir("back")
                species_rate1 = -species_rate1
            else:
                edge1.set_dir("forward")
            # Set the edge pen width
            if species_rate1 < species_rate_tol:
                penwidth = 0.0
                edge1.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
            edge1.set_penwidth(penwidth)
            if label1 and label_automatically:
                edge1.set_decorate(True)
                edge1.set_label(label1)

            graph.add_edge(edge1)

            # add mech 2
            if species1_to_2[reactant_index1] >= 0 and species1_to_2[product_index1] >= 0:
                reactant_index2 = species1_to_2[reactant_index1]
                product_index2 = species1_to_2[product_index1]

                edge2 = pydot.Edge(str(reactant1), str(product1), color=color2)
                species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate_total
                if species_rate2 < 0:
                    edge2.set_dir("back")
                    species_rate2 = -species_rate2
                else:
                    edge2.set_dir("forward")
                # Set the edge pen width
                if species_rate2 < species_rate_tol:
                    penwidth = 0.0
                    edge2.set_dir("none")
                else:
                    penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)
                edge2.set_penwidth(penwidth)
                graph.add_edge(edge2)

    # Go through edges in Mechanism 2
    for reactant_index2, product_index2 in edges2:
        # skip if this was already done in edges 1
        if [species2_to_1[reactant_index2], species2_to_1[product_index2]] in edges1 or \
            [species2_to_1[product_index2], species2_to_1[reactant_index2]] in edges1:
            continue

        if reactant_index2 in nodes2 and product_index2 in nodes2:
            # mech 2 says include this edge for all mechs
            if species2_to_1[reactant_index2] in nodes1:
                reactant2 = species_list1[species2_to_1[reactant_index2]]
            else:
                reactant2 = species_list2[reactant_index2]
            if species2_to_1[product_index2] in nodes1:
                product2 = species_list1[species2_to_1[product_index2]]
            else:
                product2 = species_list2[product_index2]
            edge2 = pydot.Edge(str(reactant2), str(product2), color=color2)

            species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate_total
            if species_rate2 < 0:
                edge2.set_dir("back")
                species_rate2 = -species_rate2
            else:
                edge2.set_dir("forward")
            # Set the edge pen width
            if species_rate2 < species_rate_tol:
                penwidth = 0.0
                edge2.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)



            edge2.set_penwidth(penwidth)
            graph.add_edge(edge2)

            # add mech 1
            if species2_to_1[reactant_index2] >= 0 and species2_to_1[product_index2] >= 0:
                reactant_index1 = species2_to_1[reactant_index2]
                product_index1 = species2_to_1[product_index2]

                edge1 = pydot.Edge(str(reactant2), str(product2), color=color1)
                species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate_total
                if species_rate1 < 0:
                    edge1.set_dir("back")
                    species_rate1 = -species_rate1
                else:
                    edge1.set_dir("forward")
                # Set the edge pen width
                if species_rate1 < species_rate_tol:
                    penwidth = 0.0
                    edge1.set_dir("none")
                else:
                    penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
                edge1.set_penwidth(penwidth)
                label2 = ''
                if label2 and label_automatically:
                    edge1.set_decorate(True)
                    edge1.set_label(label2)
                graph.add_edge(edge1)


    # General purpose graph settings
    graph.set_nodesep(0.11)
    graph.set_ranksep(0.35)
    graph.set_rankdir('LR')

    # Add Legend
    graph.add_node(pydot.Node(mech_1_label + f'\nt={times1[t1]:.4e}', label=mech_1_label + f'\nt={times1[t1]:.4e}', color=color1, shape='box', penwidth=max_node_pen_width))
    graph.add_node(pydot.Node(mech_2_label + f'\nt={times2[t2]:.4e}', label=mech_2_label + f'\nt={times2[t2]:.4e}', color=color2, shape='box', penwidth=max_node_pen_width))


    # write in multiple formats
    # graph.write_dot(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1:04}.dot')) # Yes this is supposed to be an index, not an actual time
    graph.write_png(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1:04}.png'))
    # graph.write_pdf(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1:04}.pdf'))
    # graph.write_pdf(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1:04}.svg'))


